In [1]:
# Tokenizer imports
from transformers import DistilBertTokenizerFast

# Data handling imports
from datasets import *
import numpy as np

# General imports
import torch
import ast

In [2]:
# Load the dataset from file and split it into train and test datasets
data = load_dataset('csv', data_files="../data/clean/sustainability-report-2020-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=42)

Found cached dataset csv (C:/Users/rjutr/.cache/huggingface/datasets/csv/default-656364e83e74f69c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [3]:
# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data["test"] = data["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                  "text": example["text"], "answer_start": example["answer_start"]}})
data["train"].remove_columns(["text", "answer_start"])

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'context', 'answers', 'id'],
    num_rows: 129
})

In [4]:
# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained(
    'distilbert-base-cased-distilled-squad')

In [5]:
def tokenize_sample_data(data):
    # Tokenize the data
    tokenized_feature = tokenizer(
        data["question"],
        data["context"],
        max_length=384,
        return_overflowing_tokens=True,
        stride=128,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
    )

    # When it overflows, multiple rows will be returned for a single example.
    # The following then gets the array of corresponding the original sample index.
    sample_mapping = tokenized_feature.pop("overflow_to_sample_mapping")
    # Get the array of [start_char, end_char + 1] in each token.
    # The shape is [returned_row_size, max_length]
    offset_mapping = tokenized_feature.pop("offset_mapping")

    start_positions = []
    end_positions = []
    for i, offset in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        answers = data["answers"][sample_index]
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0]) - 1
        # The format of sequence_ids is [None, 0, ..., 0, None, None, 1, ..., 1, None, None, ...]
        # in which question's token is 0 and contex's token is 1
        sequence_ids = tokenized_feature.sequence_ids(i)
        # find the start and end index of context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        # Set start positions and end positions in inputs_ids
        # Note: The second element in offset is end_char + 1
        # if offset[context_start][0] > end_char or offset[context_end][1] <= start_char:
        if not (offset[context_start][0] <= start_char and end_char < offset[context_end][1]):
            # The case that answer is not inside the context
            # Note : Some tokenizer (such as, tokenizer in rinna model) doesn't place CLS
            # for the first token in sequence, and I then set -1 as positions.
            # (Later I'll process rows with start_positions=-1.)
            start_positions.append(-1)
            end_positions.append(-1)
        else:
            # The case that answer is found in the context

            # Set start position
            idx = context_start
            while offset[idx][0] < start_char:
                idx += 1
            if offset[idx][0] == start_char:
                start_positions.append(idx)
            else:
                start_positions.append(idx - 1)

            # Set end position
            idx = context_end
            while offset[idx][1] > end_char + 1:
                idx -= 1
            if offset[idx][1] == end_char + 1:
                end_positions.append(idx)
            else:
                end_positions.append(idx + 1)

    # Build result
    tokenized_feature["start_positions"] = start_positions
    tokenized_feature["end_positions"] = end_positions
    return tokenized_feature


# Run conversion
tokenized_ds = data.map(
    tokenize_sample_data,
    remove_columns=["id", "context", "question", "answers"],
    batched=True,
    batch_size=128)

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [6]:
# Remove rows with no answer
tokenized_ds = tokenized_ds.filter(lambda x: x["start_positions"] != -1)

Filter:   0%|          | 0/129 [00:00<?, ? examples/s]

Filter:   0%|          | 0/56 [00:00<?, ? examples/s]

In [7]:
# QA model import
from transformers import AutoModelForQuestionAnswering

# Set the device
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load the model
model = AutoModelForQuestionAnswering.from_pretrained(
    "distilbert-base-cased-distilled-squad").to(device)

In [8]:
# Optimizer import
from transformers import DefaultDataCollator

# Set the data collator
data_collator = DefaultDataCollator()

In [9]:
# Trainer import
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="distilbert-nlb-qa",
    log_level="error",
    num_train_epochs=10,
    learning_rate=2e-5,
    lr_scheduler_type="linear",
    warmup_steps=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=16,
    evaluation_strategy="steps",
    eval_steps=2,
    save_steps=2,
    logging_steps=2,
    push_to_hub=False
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
)

In [10]:
# Fine-tune the model
trainer.train()

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

{'loss': 0.4482, 'learning_rate': 2e-05, 'epoch': 2.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.6986376047134399, 'eval_runtime': 1.7126, 'eval_samples_per_second': 32.7, 'eval_steps_per_second': 2.336, 'epoch': 2.0}
{'loss': 0.3528, 'learning_rate': 1.5000000000000002e-05, 'epoch': 4.0}


...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.6124652028083801, 'eval_runtime': 0.449, 'eval_samples_per_second': 124.723, 'eval_steps_per_second': 8.909, 'epoch': 8.0}
{'loss': 0.1979, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.609766960144043, 'eval_runtime': 1.4778, 'eval_samples_per_second': 37.894, 'eval_steps_per_second': 2.707, 'epoch': 10.0}
{'train_runtime': 62.7823, 'train_samples_per_second': 20.547, 'train_steps_per_second': 0.159, 'train_loss': 0.2952278912067413, 'epoch': 10.0}


TrainOutput(global_step=10, training_loss=0.2952278912067413, metrics={'train_runtime': 62.7823, 'train_samples_per_second': 20.547, 'train_steps_per_second': 0.159, 'train_loss': 0.2952278912067413, 'epoch': 10.0})

In [11]:
# Evaluate the model
trainer.evaluate()

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.609766960144043,
 'eval_runtime': 1.4478,
 'eval_samples_per_second': 38.678,
 'eval_steps_per_second': 2.763,
 'epoch': 10.0}

In [12]:
# Define the prediction function
def inference_answer(model, question, context):
    question = question
    context = context
    test_feature = tokenizer(
        question,
        context,
        max_length=318
    )
    with torch.no_grad():
        outputs = model(torch.tensor([test_feature["input_ids"]]).to(device))
    start_logits = outputs.start_logits.cpu().numpy()
    end_logits = outputs.end_logits.cpu().numpy()
    answer_ids = test_feature["input_ids"][np.argmax(
        start_logits):np.argmax(end_logits)+1]
    return " ".join(tokenizer.batch_decode(answer_ids))

In [13]:
# Generate an array of predictions and an array of true answers
answer_pred = [inference_answer(model, data["test"]["question"][idx],
                                data["test"]["context"][idx]) for idx in range(data["test"].shape[0])]
answer_true = [data["test"]["answers"][idx]["text"][0]
               for idx in range(data["test"].shape[0])]

In [14]:
# Importing the evaluation library
import evaluate
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=answer_pred,
                            references=answer_true, lang="en")
# Embeddings bases evaluation
print(
    f"F1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

F1: 0.9159998670220375, Precision: 0.9026597280587468, Recall: 0.9308400345700127


In [15]:
# SQuAD v2.0 evaluation
squad_v2_metric = evaluate.load("squad_v2")
references = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [
    answer["text"][0]]}, "id": str(id)} for id, answer in zip(data["test"]["id"], data["test"]["answers"])]
predictions = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.}
               for id, answer in zip(data["test"]["id"], answer_pred)]
results = squad_v2_metric.compute(
    predictions=predictions, references=references)
results

{'exact': 42.857142857142854,
 'f1': 55.69200102002897,
 'total': 56,
 'HasAns_exact': 42.857142857142854,
 'HasAns_f1': 55.69200102002897,
 'HasAns_total': 56,
 'best_exact': 42.857142857142854,
 'best_exact_thresh': 0.0,
 'best_f1': 55.69200102002897,
 'best_f1_thresh': 0.0}

In [16]:
bleu = evaluate.load("bleu")
references = [[answer] for answer in answer_true]
predictions = answer_pred
# N-Gram based evaluation
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.1493326152181526,
 'precisions': [0.2894736842105263,
  0.1888111888111888,
  0.11885245901639344,
  0.07655502392344497],
 'brevity_penalty': 1.0,
 'length_ratio': 2.206451612903226,
 'translation_length': 342,
 'reference_length': 155}

In [17]:
# Load the same model without fine-tuning on our dataset
model_no_ft = AutoModelForQuestionAnswering.from_pretrained(
    "distilbert-base-cased-distilled-squad").to(device)

In [18]:
# Generate another set of predictions
answer_pred_no_ft = [inference_answer(model_no_ft, data["test"]["question"]
                                      [idx], data["test"]["context"][idx]) for idx in range(data["test"].shape[0])]

In [19]:
results = bertscore.compute(
    predictions=answer_pred_no_ft, references=answer_true, lang="en")
# Embeddings bases evaluation
print(
    f"F1: {np.array(results['f1']).mean()}, Precision: {np.array(results['precision']).mean()}, Recall: {np.array(results['recall']).mean()}")

F1: 0.9190749110920089, Precision: 0.9059299964989934, Recall: 0.933719420007297


In [20]:
references = [{"answers": {"answer_start": [answer["answer_start"][0]], "text": [
    answer["text"][0]]}, "id": str(id)} for id, answer in zip(data["test"]["id"], data["test"]["answers"])]
predictions = [{"id": str(id), "prediction_text": answer, "no_answer_probability": 0.}
               for id, answer in zip(data["test"]["id"], answer_pred_no_ft)]
# SQuAD v2.0 evaluation
results = squad_v2_metric.compute(
    predictions=predictions, references=references)
results

{'exact': 44.642857142857146,
 'f1': 57.656286734314676,
 'total': 56,
 'HasAns_exact': 44.642857142857146,
 'HasAns_f1': 57.656286734314676,
 'HasAns_total': 56,
 'best_exact': 44.642857142857146,
 'best_exact_thresh': 0.0,
 'best_f1': 57.656286734314676,
 'best_f1_thresh': 0.0}

In [21]:
references = [[answer] for answer in answer_true]
predictions = answer_pred_no_ft
# N-Gram based evaluation
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.15141296248186284,
 'precisions': [0.29705882352941176,
  0.1936619718309859,
  0.11934156378600823,
  0.07655502392344497],
 'brevity_penalty': 1.0,
 'length_ratio': 2.193548387096774,
 'translation_length': 340,
 'reference_length': 155}

In [22]:
# Locally save the model
trainer.save_model("./distilbert-nlb-qa")